In [1]:
import pandas as pd
import pd_explain

usetex-False
(CVXPY) Dec 23 04:28:59 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Dec 23 04:28:59 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
adults = pd.read_csv(r"..\Examples\Datasets\adult.csv")

# Using the many to one explainer on the entire dataset:

Using the many to one explainer on a column from the dataset that has a many to one relationship with other columns (the visualization uses PCA for dimensionality reduction):

In [3]:
adults.explain(explainer='many_to_one', labels='label', coverage_threshold=0.8, separation_threshold=0.2)

# Using the many to one explainer on a subset of the dataset:

Using the many to one explainer on subsets of the data, that only contain numerical / categorical columns:

In [4]:
adults_numeric_only = adults[['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]
adults_numeric_only

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40
...,...,...,...,...,...,...
48837,39,215419,13,0,0,36
48838,64,321403,9,0,0,40
48839,38,374983,13,0,0,50
48840,44,83891,13,5455,0,40


Just like we could provide a column name (if it is a column in the dataset), we can also pass a pandas series as the labels parameter:

In [4]:
labels = adults['label']

In [6]:
adults_numeric_only.explain(explainer='many_to_one', labels=labels, coverage_threshold=0.8, separation_threshold=0.2)

In [7]:
adults_categorical_only = adults[['workclass', 'education', 'marital-status', 'occupation', 'relationship']]
adults_categorical_only

,workclass,education,marital-status,occupation,relationship
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife
...,...,...,...,...,...
48837,Private,Bachelors,Divorced,Prof-specialty,Not-in-family
48838,?,HS-grad,Widowed,?,Other-relative
48839,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband
48840,Private,Bachelors,Divorced,Adm-clerical,Own-child


In [8]:
adults_categorical_only.explain(explainer='many_to_one', labels=labels, coverage_threshold=0.8, separation_threshold=0.2)

We can also specify only specific columns to be used in the many to one explainer:

In [4]:
adults.explain(explainer='many_to_one', select_columns=['age', 'education', 'marital-status', 'relationship'], labels='label', coverage_threshold=0.8, separation_threshold=0.5)

# Using many to one explainer with clustering labels:

In [5]:
from sklearn.cluster import KMeans

In [6]:
adults_with_dummies = pd.get_dummies(adults)

In [ ]:
cluster_labels = KMeans(n_clusters=5).fit_predict(adults_with_dummies)

In [9]:
adults.explain(explainer='many_to_one', labels=cluster_labels, coverage_threshold=0.8, separation_threshold=0.2)

# Using many to one explainer to explain groupby groups:

Many to one explainer can also be used to explain groupby groups. It can work with single or multiple attributes groupby:

In [13]:
single_attribute_groupby = adults.groupby('relationship').mean()
single_attribute_groupby

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
relationship,,,,,,
Husband,43.906066,187193.324102,10.305133,1774.499797,122.200649,44.166109
Not-in-family,38.421601,190352.245013,10.326870,783.963284,76.840817,40.529842
Other-relative,33.420983,203524.602258,8.849270,234.989376,49.920983,37.128154
Own-child,24.794090,193782.299697,9.494394,174.381480,41.276217,33.142461
Unmarried,40.312195,191370.284878,9.631610,450.863805,46.638634,39.172488
Wife,40.078507,180748.781639,10.488631,1658.779923,116.033033,36.729730


To explain a groupby's groups, we can use the explainer on the groupby dataframe without specifying the labels parameter:

In [15]:
single_attribute_groupby.explain(explainer='many_to_one', coverage_threshold=0.8, separation_threshold=0.2)

In [18]:
multi_attribute_groupby = adults.groupby(['sex', 'label']).mean()
multi_attribute_groupby

age         fnlwgt  education-num  capital-gain  \
sex    label                                                          
Female <=50K  36.278999  185690.282604       9.824724    122.876586   
       >50K   42.219333  183989.519503      11.832109   4313.663652   
Male   <=50K  37.248548  192799.098671       9.454953    162.322673   
       >50K   44.641863  189269.827082      11.561908   3993.827586   

              capital-loss  hours-per-week  
sex    label                                
Female <=50K     46.910976       35.875962  
       >50K     180.222725       40.678915  
Male   <=50K     58.746173       40.720702  
       >50K     195.902299       46.304396

In [19]:
multi_attribute_groupby.explain(explainer='many_to_one', coverage_threshold=0.8, separation_threshold=0.55)

# Many to one explainer when there are many clusters:

In [20]:
from sklearn.cluster import KMeans
adults_with_dummies = pd.get_dummies(adults)

In [23]:
many_clusters = KMeans(n_clusters=11).fit_predict(adults_with_dummies)

In the case there are many clusters (over 10), the layout of the plot will be different:

In [24]:
adults.explain(explainer='many_to_one', labels=many_clusters, coverage_threshold=0.8, separation_threshold=0.2)

# 3d visualization:

Many to one explainer supports both 2d and 3d visualizations:

In [3]:
adults.explain(explainer='many_to_one', labels='label', coverage_threshold=0.8, separation_threshold=0.2, visualization_dims=3)